In [1]:
from create_dataloaders import *
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim

C:\Users\Dani\anaconda3\envs\onlab\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class OneDConvNet(nn.Module):

    def __init__(self, n_features):
        super(OneDConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv1d(in_channels=n_features, out_channels = 256, kernel_size=3, padding = 'same'),
            nn.ReLU(),
            nn.BatchNorm1d(num_features = 256)
            #nn.Dropout(0.5)
        )
        self.layer2 = nn.Sequential(
            nn.Conv1d(in_channels=256, out_channels = 256, kernel_size=3, padding = 'same'),
            nn.ReLU(),
            nn.BatchNorm1d(num_features = 256)
            #nn.Dropout(0.5)
        )
        self.layer3 = nn.Sequential(
            nn.Conv1d(in_channels=256, out_channels = 128, kernel_size=3, padding = 'same'),
            nn.ReLU(),
            nn.BatchNorm1d(num_features = 128)
            #nn.Dropout(0.5)
        )
        self.layer4 = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels = 64, kernel_size=3, padding = 'same'),
            nn.ReLU(),
            nn.BatchNorm1d(num_features = 64)
            #nn.Dropout(0.5)
        )
        self.layer5 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels = 32, kernel_size=3, padding = 'same'),
            nn.ReLU(),
            nn.BatchNorm1d(num_features = 32)
            #nn.Dropout(0.5)
        )
        self.layer6 = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels = 30, kernel_size=8),
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        return out

In [3]:
spectogram_train, spectogram_val, spectogram_test, features_train, features_val, features_test = get_data()
# Only need to be run if train_stats.json is missing
# write_statistics_to_json(features_train)
train_dataset, val_dataset, test_dataset = create_datasets(spectogram_train, spectogram_val, spectogram_test, features_train, features_val, features_test, window=30)
train_dl,val_dl,test_dl = create_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=32)

In [7]:
print(len(train_dataset))
print(len(train_dl))
epoch = 10
window_size = 30
batch_size = 32
learning_rate = 0.01
if torch.cuda.is_available():
    print("CUDA available")
    device = "cuda:0"
    device = torch.device(device)
num_features = 4860

47946
1499
CUDA available


In [5]:
model = OneDConvNet(num_features)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))
model.to(device)

OneDConvNet(
  (layer1): Sequential(
    (0): Conv1d(4860, 256, kernel_size=(3,), stride=(1,), padding=same)
    (1): ReLU()
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=same)
    (1): ReLU()
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer3): Sequential(
    (0): Conv1d(256, 128, kernel_size=(3,), stride=(1,), padding=same)
    (1): ReLU()
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer4): Sequential(
    (0): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=same)
    (1): ReLU()
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer5): Sequential(
    (0): Conv1d(64, 32, kernel_size=(3,), stride=(1,), padding=same)
    (1): ReLU()
    (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=Tr

In [ ]:
for epoch in range(1,epoch+1):  # loop over the dataset multiple times
    i=0
    sum_loss = 0.0
    val_loss = 0.0
    for inputs, labels in tqdm(train_dl, 0):
        i = i+1
        inputs = torch.permute(inputs, (0, 2, 1))
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        sum_loss += loss.item()

    if epoch % 1 == 0:
        print(f"Epoch: {epoch}, avg training loss: {sum_loss/len(train_dl)}") 
    
    for in_v, target_v in val_dl:
            in_v= torch.permute(in_v, (0, 2, 1))
            in_v = in_v.to(device)
            target_v = target_v.to(device)
            outputs = model.forward(in_v)
            v_loss = criterion(outputs, target_v)
            val_loss += v_loss.item()
            
    print(f"Validation loss: {v_loss.item()/len(val_dl)}")
            


100%|██████████████████████████████████████████████████████████████████████████████| 1499/1499 [03:48<00:00,  6.56it/s]


Epoch: 1, avg training loss: 1.4600196856113177
Validation loss: 0.02615857893420804


100%|██████████████████████████████████████████████████████████████████████████████| 1499/1499 [03:51<00:00,  6.47it/s]


Epoch: 2, avg training loss: 0.8515614729232992
Validation loss: 0.021038213083820957


100%|██████████████████████████████████████████████████████████████████████████████| 1499/1499 [03:54<00:00,  6.38it/s]


Epoch: 3, avg training loss: 0.7019005198928815
Validation loss: 0.02342009288008495


100%|██████████████████████████████████████████████████████████████████████████████| 1499/1499 [03:49<00:00,  6.54it/s]


Epoch: 4, avg training loss: 0.6111875451589283
Validation loss: 0.01618587842551611


100%|██████████████████████████████████████████████████████████████████████████████| 1499/1499 [04:00<00:00,  6.24it/s]
